# Day 11

link: https://adventofcode.com/2022/day/11

ข้อนี้ค่อนข้างซับซ้อนกว่าข้อที่ผ่านๆ มา เราจะมาย่อยโจทย์กันก่อน

เราสามารถมองลิงแต่ละตัว เป็น function ที่รับ input เข้ามาเป็น integer แล้วส่ง output เป็น integer และลิงที่จะต้องส่งต่อให้ อาจจะ define signature ได้ว่า `Integer -> (Integer, Monkey)`

รายละเอียดของ function จะแตกต่างกันไป แต่โดยรวมแล้วมันมี step เหมือนๆ กัน คือ
1. apply operation ซึ่งอาจจะเป็นการบวกหรือการคูณ เข้ากับ input integer
2. stress relieve เป็นอีก operation นึง ซึ่งจะลดผลลัพธ์จากข้อ 1 ลงมา
3. test ว่าผลจากข้อ 2 หารด้วยจำนวนที่กำหนด (ซึ่งเป็นจำนวนเฉพาะ) ลงตัวหรือไม่
4. ถ้าหารลงตัว จะส่งผลจากข้อ 2 ไปให้ลิงตัวหนึ่ง ถ้าหารไม่ลงตัว ก็จะส่งไปให้อีกตัวหนึ่ง

โจทย์สนใจว่าในแต่ละรอบ ลิงแต่ละตัว process ของไปกี่ชิ้น
ตรงรายละเอียดที่บอกว่า ต้อง process ของตามลำดับ อันนี้เราไม่ต้องสนใจก็ได้
ยังไงซะลิงก็จะต้อง process ของทุกชิ้นอยู่ดี ไม่ว่าจะตามลำดับหรือไม่ ได้ผลไม่ต่างกัน
ดังนั้นแทนที่จะต้องเก็บ list ของของที่ต้อง process เรา track ของแต่ละชิ้นเลยก็ได้ แล้วนับจบรอบเมื่อของถูกส่งจากลิงที่เลขลำดับสูงกว่าไปยังต่ำกว่า

In [1]:
class Monkey
  attr_accessor  :id, :op, :divisor, :true_target, :false_target, :items_inspected, :relieve

  def initialize(id)
    @id = id
    @items_inspected = 0
  end

  def exec_once(item)
    old = item
    item = @relieve.call(@op.call(item))
    pass_test = (item % @divisor) == 0
    target = pass_test ? @true_target : @false_target
    @items_inspected += 1
    [item, target]
  end

end
nil

ส่วนของการ parse input มีความยุ่งยากนิดหน่อยตรง operation ซึ่งเราต้องการเก็บเป็น lambda ท่าแรกที่เราลองคือใช้ `eval` ซึ่งก็ใช้ได้ดี
แต่ระหว่างทดลองรันเรารู้สึกว่ามันช้าไปหน่อย เลยเปลี่ยนมา parse จากรูปแบบ `operand1 operation operand2` ซึ่งเร็วขึ้นแต่ก็ rigid กว่า
ก็ไม่เป็นไร เราไม่ได้ต้องการ flexibility เท่าไหร่

สังเกตว่าผลของการ parse เราจะยังไม่ได้ตั้งค่าให้ `:relieve` เพราะว่ามันเป็นรายละเอียดที่ต่างกันของโจทย์แต่ละ part

In [2]:
input = IO.foreach('data/11.txt').to_a.map(&:strip)

def parse_monkeys(input)
  monkeys = []
  start_items = []
  current_monkey = nil
  input.each{|line|
    if line.start_with?('Monkey')
      current_monkey = Monkey.new(line.split(' ')[-1].to_i)
      monkeys[current_monkey.id] = current_monkey
    elsif line.strip.start_with?('Starting items')
      start_items[current_monkey.id] = line.strip.split(':')[1].split(',').map(&:to_i)
    else
      cmd, args = line.split(':')
      case cmd
      when 'Operation'
        arg1, op, arg2 = args.split('=')[1].split(' ')
        current_monkey.op = ->(x){
          a = arg1 == 'old' ? x : arg1.to_i
          b = arg2 == 'old' ? x : arg2.to_i
          a.send(op.to_sym, b)
        }
      when 'Test'
        if args.start_with?(' divisible by')
          current_monkey.divisor = args.split(' ')[-1].to_i
        end
      when 'If true'
        current_monkey.true_target = args.split(' ')[-1].to_i
      when 'If false'
        current_monkey.false_target = args.split(' ')[-1].to_i
      end
    end
  }
  [monkeys, start_items]
end
nil

## Part 1

part แรกนี่ตรงไปตรงมา พอ parse ลิงเสร็จก็เหลือแค่ตั้ง relieve เป็น `x -> x/3` แล้ว simulate ไป 20 รอบ

In [3]:
def simulate(monkeys, start_items, rounds)
  start_items.size.times{|i|
    start_items[i].each{|item|
      m = i
      round = 0
      while round < rounds
        item, new_m = monkeys[m].exec_once(item)
        round += 1 if new_m < m
        m = new_m
      end
    }
  }
end

monkeys, start_items = parse_monkeys(input)
relieve = ->(x){x / 3}
monkeys.each{|monkey| monkey.relieve = relieve}
simulate(monkeys, start_items, 20)

v1, v2 = monkeys.map(&:items_inspected).sort[-2..-1]
puts v1 * v2

58794


## Part 2

part นี้ เป็นโจทย์แรกของปีนี้ที่ไม่บอกรายละเอียดทั้งหมด แต่ให้เราลองเดาเอาจากตัวอย่าง

โจทย์บอกว่าเราจะเปลี่ยนวิธี relieve แต่ไม่บอกว่าเปลี่ยนเป็นอะไร แค่ใบ้ๆ ว่า ถ้าไม่มี relieve แล้ว
worry level มันจะพุ่งนะ

ซึ่ง... เราก็ไม่ได้สนใจ hint ตรงนี้ทีแรก ก็เลยไปลองตั้ง relieve ให้เป็นการลบด้วย constant
ซึ่งก็สามารถหา constant ที่ทำให้ result ตรงกับตัวอย่างใน 20 รอบแรกได้อยู่
แต่พอจะลองที่ 1000 รอบ ปรากฏว่า mem ไม่พอ 

ตอนแรกเราก็นึกว่ามีปัญหาที่ `eval` หรือ lambda หรือเปล่า แต่เมื่อ debug ดูก็พบว่า ตัวเลขมันใหญ่จริงๆ จนเกิน memory limit!
อันนี้ก็ทำให้กลับมาตั้งใจดู hint (ที่จริงการที่มัน out of memory ก็เป็น hint อย่างนึงเหมือนกัน)
เราต้องหาวิธีที่ทำให้ตัวเลขมันไม่ใหญ่เกินไป ซึ่งการลบอย่างเดียวอาจจะลดได้น้อยเกินไป

แน่นอนว่าก็ต้องใช้ modular arithmetic มาช่วย สังเกตดูว่า operation ที่ใช้ทั้งหมด ใช้ operator แค่ บวก ลบ คูณ และ เช็คว่าหารลงตัวหรือไม่
(ไม่นับ floor division ด้วย 3 จาก Part แรก) ซึ่งทั้งหมดนี้เป็น operation ที่ยังทำงานได้เหมือนเดิมใน (mod m)
ถ้าให้ m = Product of all test divisors

ถึงตรงนี้เราเลยเปลี่ยน relieve เป็น `x -> (x - k) % m` แล้วลองหา k ที่ให้ผลลัพธ์ตรงกับตัวอย่าง
ก็ไปเจอคำตอบที่ k = m ... ซึ่งใน (mod m) มันคือ k = 0 นั่นเอง

สรุปว่า relieve จริงๆ มันคือ `x -> x % m` แค่นี้เอง หาอยู่ตั้งนาน ปัดโธ่

In [4]:
monkeys, start_items = parse_monkeys(input)
m = monkeys.map(&:divisor).reduce(:*)
relieve = ->(x){x % m}
monkeys.each{|monkey| monkey.relieve = relieve}
simulate(monkeys, start_items, 10000)

v1, v2 = monkeys.map(&:items_inspected).sort[-2..-1]
puts v1 * v2

20151213744
